<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Finance Basics

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Reinforcement Learning

In [ ]:
!git clone https://github.com/tpq-classes/pff_basics.git
import sys
sys.path.append('pff_basics')


In [ ]:
import numpy as np
import pandas as pd
from pylab import plt
np.set_printoptions(suppress=True)
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

## `CartPole` Game

**Environment & Agents**

Topics:

**Reinforcement Learning**

* environment
* state
* agent
* action
* step
* reward/penalty
* objective
* policy
* episode


**Deep Q-Learning**

* reward function
* action policy
* representation
* deep neural network
* exploration/exploitation
* replay & policy update


## Gym Environment

In [ ]:
import gymnasium as gym

In [ ]:
env = gym.make('CartPole-v1')

In [ ]:
env.observation_space

In [ ]:
env.action_space

In [ ]:
env.action_space.n

In [ ]:
env.action_space.contains(0)

In [ ]:
env.action_space.contains(1)

In [ ]:
for _ in range(10):
    print(env.action_space.sample(), end=' ')

In [ ]:
a = env.action_space.sample()
a

In [ ]:
env.reset()

In [ ]:
env.step(a)

In [ ]:
env.step(0)

## DQL Agent

In [ ]:
import random
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
class DQLAgent:
    def __init__(self):
        self.epsilon = 1.0
        self.epsilon_decay = 0.99
        self.epsilon_min = 0.1
        self.memory = list()
        self.batch_size = 32
        self.gamma = 0.95
        self._create_model()
    def _create_model(self):
        self.model = Sequential()
        self.model.add(Dense(24, activation='relu', input_dim=4))
        self.model.add(Dense(24, activation='relu'))
        self.model.add(Dense(2, activation='linear'))
        self.model.compile(loss='mse', optimizer='adam')
    def act(self, state):
        if random.random() < self.epsilon:
            return env.action_space.sample()
        return np.argmax(self.model.predict(state)[0])
    def replay(self):
        batch = random.sample(self.memory, self.batch_size)
        for state, action, next_state, reward, done in batch:
            if not done:
                reward += self.gamma * np.amax(self.model.predict(next_state)[0])
            target = self.model.predict(state)
            target[0, action] = reward
            # print(reward, target)
            self.model.fit(state, target, epochs=1, verbose=False)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    def learn(self, episodes):
        for e in range(1, episodes + 1):
            state, _ = env.reset()
            state = np.reshape(state, [1, 4])
            for f in range(1, 201):
                action = self.act(state)
                next_state, reward, done, trunc, _ = env.step(action)
                next_state = np.reshape(next_state, [1, 4])
                self.memory.append([state, action, next_state, reward, done])
                state = next_state
                if done:
                    print(f, end=' ')
                    break
            if len(self.memory) > self.batch_size:
                self.replay()

In [ ]:
agent = DQLAgent()

In [ ]:
# agent.act()

In [ ]:
agent.learn(50)

In [ ]:
agent.epsilon

In [ ]:
# agent.memory

In [ ]:
# agent.replay()

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="mailto:training@tpq.io">training@tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> 